Importation of modules

In [9]:
import os
# import matplotlib.pylab as plt
import numpy as np
import xarray as xr
# from scipy import signal
from obspy.signal.filter import bandpass

Specific modules

In [1]:
from tools_pcc import *

In [2]:
from stack_functions_new import *

In [3]:
from xdas.io.febus import read as read_das
from xdas.io.febus import correct_gps_time

Paths

In [4]:
user = "mba"

if user == "ipgp" :
    path1 = "/home/hugonnet/Documents/DAS-Lucie/fichiers-nc/"

elif user == "mba" :
    path1 = "/Volumes/DISKENOIR/fichiers-nc/"
    path2 = "/Volumes/DISKENOIR/Correlograms/Cross-correlation/" # storage of correlograms for cross-correlation
    path3 = "/Volumes/DISKENOIR/Correlograms/Auto-correlation/" # storage of correlograms for auto-correlation

elif user == "eleonore" :
    path1 = '/Users/stutz/DAS/DAS_STROMBOLI/2022_STROMBOLI_MANIP/SILIXA_iDAS/DATA_VLP_FEBUS/'

In [5]:
file_name = 'SR_FEBUS_STROMBOLI_2022-09-18_17-35-44_UTC.nc'

Definition of the parameters

In [16]:
corr_type = 'phase-cross'   # can be 'phase-cross' (cross-correlation), 'phase-auto' (auto-correlation), 'normal', or 'onebit'

# define variable for correlation
window = 300 			# duration of time window in second
lag = window/2 			# shift of starting time on the next iteration
time = 0 			# time initialization

# filter boundary (Hz)
lowcut = 1
highcut = 10

start_channel = 300		# starting/reference point. currently used: 100, 200, 320, 400
nb_channel = 150		# number of channel used			 
start_position = 'mid_all' 

change_gauge_length = False
gauge_length = 50

In [17]:
if corr_type == 'phase-cross' :
    folder_destination = path2 + file_name.replace(".nc", "/") # please make the folder before running

elif corr_type == 'phase-auto' :
    folder_destination = path3 + file_name.replace(".nc", "/") # please make the folder before running

In [18]:
ds = xr.open_dataset(path1 + file_name)
channels_coord = ds['channels']

In [19]:
ds

<xarray.Dataset>
Dimensions:       (times: 180000, offsets: 3250, channels: 3250)
Coordinates:
  * times         (times) datetime64[ns] 2022-09-18T17:35:44.500000 ... 2022-...
  * offsets       (offsets) float64 0.0 2.0 4.0 ... 6.496e+03 6.498e+03
  * channels      (channels) float64 0.0 1.0 2.0 ... 3.248e+03 3.249e+03
Data variables:
    strain_rates  (times, offsets) float32 ...
Attributes:
    unit:     strain rates in 10e-8 /s

In [20]:
iDas_nc = xr.DataArray(
    data=ds['strain_rates'],  # Remplacez par les données réelles de 'strain_rates'
    dims=("times", "channels"),  # Spécifiez les dimensions
    coords={"times": ds['strain_rates'].coords["times"], "channels": channels_coord},
)

In [21]:
iDas_nc

<xarray.DataArray 'strain_rates' (times: 180000, channels: 3250)>
array([[ 0.0000000e+00,  0.0000000e+00, -3.4953697e-08, ...,
         4.4692129e+03, -8.0042388e-05,  5.1425472e-05],
       [ 0.0000000e+00,  0.0000000e+00,  6.1442131e-09, ...,
         2.8822549e+03, -5.6834502e-05,  3.3164910e-05],
       [ 0.0000000e+00,  0.0000000e+00, -1.7207492e-08, ...,
         3.8322634e+03, -7.1476410e-05,  4.4096316e-05],
       ...,
       [ 0.0000000e+00, -8.2707105e-12,  1.3172808e-08, ...,
         1.0095570e+02, -1.7541629e-06,  1.1616555e-06],
       [ 0.0000000e+00,  0.0000000e+00,  7.6554487e-09, ...,
         9.9110550e+01, -1.7135526e-06,  1.1404245e-06],
       [ 0.0000000e+00,  0.0000000e+00, -3.6883813e-10, ...,
         9.9572304e+01, -1.7854875e-06,  1.1457379e-06]], dtype=float32)
Coordinates:
  * times     (times) datetime64[ns] 2022-09-18T17:35:44.500000 ... 2022-09-1...
  * channels  (channels) float64 0.0 1.0 2.0 ... 3.247e+03 3.248e+03 3.249e+03

Time step (dt) and sampling rate (fs)

In [22]:
dt = ( iDas_nc['times'][1].item() - iDas_nc['times'][0].item() ) * 1e-9 # s
fs = 1/dt # Hz

Correlation

In [23]:
f = open('error.txt','w') # files with few channels are not used and are stored into the file 'f'

if iDas_nc.shape[1] < 463: 
    print(f'This file only has {iDas_nc.shape[1]} channels. Skipping this file')
    f.write(f'{iDas_nc.shape[1]} channels\n')
	# skipping file if the channel is not complete

else :

    print(f'Filtering using bandpass filter between {lowcut} and {highcut} Hz ...')

    if change_gauge_length == False:
        traces_filt = iDas_nc.copy()
        print('no gauge length change')
    else:
        
        # to try GL = 10
        traces_filt = iDas_nc.copy()
        traces_filt[:,0] = iDas_nc[:,0] + iDas_nc[:,1] # the first channel with gl = 10 is defined as channel 0 + channel 1
        traces_filt[:,-1] = iDas_nc[:,-2] + iDas_nc[:,-1] # the last channel with gl = 10 is defined as channel N-1 + channel N
        print(f'Converting data to using gauge length = 10')
        #for i in range(1,traces_filt.shape[1]-2): # each end is skipped because it has been calculated in the previous line
        for i_channel in range(57, iDas_nc['channels'].values[-2]+1):
            traces_filt[:,i_channel] = iDas_nc[:,i_channel-1] + iDas_nc[:,i_channel] + iDas_nc[:,i_channel+1]
        traces_filt_gl10 = traces_filt[::2]


    # for j in [56,3249]:
    # for j in range (56,431):
    #     try:
    #         #traces_filt[:,j] = filter_bandpass(iDas_nc[:,j], lowcut, highcut, fs,order=4) # filter defined in stack_functions and using scipy.signal.butter filter
    #         traces_filt[:,j] = bandpass(iDas_nc[:,j],lowcut,highcut,fs, zerophase=True) # obspy butterworth filter (faster)
    #     except:
    #         print('filtrage pas abouti pour j = ', j)
    #         #None
		
    length = iDas_nc.shape[0]	# total length of data

	# cross correlate

    if corr_type == 'phase-cross' :
        print('\nStarting the phase cross-correlation  ...')
        pcc, t = cross_correlate(traces_filt,folder_destination,window,lag,length,time,nb_channel,
                                      start_channel,start_position,onebit=False,phase_cc=True,decimate=1,resample=1)

    elif corr_type == 'phase-auto' :
        print('\nStarting the phase auto-correlation  ...')
        pcc_auto, t = auto_correlate(traces_filt,folder_destination,window,lag,length,time,nb_channel,
                                      start_channel,start_position,onebit=False,phase_cc=True,decimate=1,resample=1)


    #print('\nStarting the normal cross-correlation  ...')
    #cc_normal, t = cross_correlate(traces_filt,window,lag,length,time,nb_channel,
    #		start_channel,start_position,onebit=False,phase_cc=False,decimate=decimate,resample=resample)

    #cc_onebit, t = cross_correlate(traces_filt,window,lag,length,time,nb_channel,
    #		start_channel,start_position,onebit=True,phase_cc=False,decimate=decimate,resample=resample)


f.close()

Filtering using bandpass filter between 1 and 10 Hz ...
no gauge length change

Starting the phase cross-correlation  ...
Start of the 1th time window. Starttime = 0.0 s
Start of the 2th time window. Starttime = 150.0 s
Start of the 3th time window. Starttime = 300.0 s
Start of the 4th time window. Starttime = 450.0 s
Start of the 5th time window. Starttime = 600.0 s
Start of the 6th time window. Starttime = 750.0 s


In [16]:
pcc

array([[-4.66666656e-04, -4.66666666e-04, -5.99999999e-04, ...,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
       [-8.00000363e-04, -9.33332969e-04, -1.19999956e-03, ...,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
       [-1.26666323e-03, -1.40000245e-03, -1.80000217e-03, ...,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
       ...,
       [-8.00000275e-04, -9.33333608e-04, -1.19999875e-03, ...,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
       [-3.33333431e-04, -4.66666764e-04, -6.00000037e-04, ...,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
       [ 2.79480143e-17, -1.96583490e-17, -5.80276568e-18, ...,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00]])

In [15]:
pcc_auto

array([[ 1.40000000e-03,  1.53333333e-03,  1.53333333e-03, ...,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
       [ 3.06666606e-03,  3.06666644e-03,  3.06666641e-03, ...,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
       [ 4.20000738e-03,  4.59999939e-03,  4.59999891e-03, ...,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
       ...,
       [ 3.06666606e-03,  3.06666644e-03,  3.06666641e-03, ...,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
       [ 1.40000000e-03,  1.53333333e-03,  1.53333333e-03, ...,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
       [-1.70530257e-17,  7.05805784e-17,  8.52651283e-18, ...,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00]])

tf-PWS (time-frequency domain Phase Weighted Stack)

In [18]:
corr_type = 'phase-cross'

In [19]:
if corr_type == 'phase-cross' :
    folder = path2 + file_name.replace(".nc", "/")

elif corr_type == 'phase-auto' :
    folder = path3 + file_name.replace(".nc", "/")

In [20]:
for i_channel in range(56,430+1):
#for i in range(100,103):

	os.system(f'cp /Users/lucie/Dropbox/MINES_3A/Stage-IPGP/DAS/Codes/Sergi-ts-PWS/src/ts_pws {folder}/Correlogram_{start_channel}_{i_channel}_pcc/')
	os.chdir(f'{folder}/Correlogram_{start_channel}_{i_channel}_pcc/')
	os.system('find . -type f -empty -print -delete') # delete empty file
	print(f'tf-pws for correlogram {start_channel} and {i_channel}')
	os.system('ls --color=never *.sac > tf-pws.in')
	#os.system('ls *.sac > tf-pws.in')
	os.system('./ts_pws tf-pws.in osac="tspws_pcc"')
	# there will be a warning that says the header is corrupted, just ignore it since we just need the cross-correlogram time array, the header information is not important


tf-pws for correlogram 100 and 56

 lat=-12345.000000
 lon=-12345.000000
 lat=-12345.000000
 lon=-12345.000000
 net1=
 sta1=
 loc1=
 chn1=
 net2=
 sta2=
 loc2=
 chn2=
stla is not fine
stlo is not fine
stla is not fine
stlo is not fine
 ReadData: Found the header corrupted when reading the 100_56_0_300_20220918_1735.sac file

 lat=-12345.000000
 lon=-12345.000000
 lat=-12345.000000
 lon=-12345.000000
 net1=
 sta1=
 loc1=
 chn1=
 net2=
 sta2=
 loc2=
 chn2=
stla is not fine
stlo is not fine
stla is not fine
stlo is not fine
 wrsac: Found the header corrupted when writing the tl_tspws_pcc.sac file

 lat=-12345.000000
 lon=-12345.000000
 lat=-12345.000000
 lon=-12345.000000
 net1=
 sta1=
 loc1=
 chn1=
 net2=
 sta2=
 loc2=
 chn2=
stla is not fine
stlo is not fine
stla is not fine
stlo is not fine
 wrsac: Found the header corrupted when writing the ts_pws_tspws_pcc.sac file
tf-pws for correlogram 100 and 57

 lat=-12345.000000
 lon=-12345.000000
 lat=-12345.000000
 lon=-12345.000000
 net1=

In [21]:
from obspy import read

In [24]:
st = read('/Volumes/DISKENOIR/Correlograms/Cross-correlation/SR_FEBUS_STROMBOLI_2022-09-18_17-35-44_UTC/Correlogram_100_56_pcc/ts_pws_tspws_pcc.sac', 
          debug_headers=True)

st[0].data

array([8.6485095e-05, 8.4961466e-05, 8.3227089e-05, ..., 8.6033178e-05,
       8.7382250e-05, 8.8085450e-05], dtype=float32)